In [332]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [333]:
%autoreload 2

In [291]:
import pktypes
from pktypes import *

In [272]:
d=Deck()
#d.populate()
#d.shuffle()

In [273]:
#d.shuffle()
d.print_cards()

2s 6c 7h 8h 9h 10c Js 4h Jd 10h 10s 2c Jh 2d Ah 9d Qd 5s 4c 7c 6h 6d 8c 8s Ac 9s Kc As Ad Kd 9c 4s 6s 5d 8d 5h Kh 4d 2h Jc 3d 7s 5c Ks Qc 10d Qh 3c 7d Qs 3s 3h 


In [274]:
d.repopulate()

In [275]:
d.shuffle()

In [276]:
c=Card('J','s')

In [277]:
for c in d.deal(4):
    print(c)

5s
5c
3d
2h


In [278]:
c.value

1

In [334]:
joe = Player('sdff', 'Joe', bank=44, stack=333)
fred = Player('sdf2', 'Fred', bank=44, stack=333)
willis = Player('sdf3', 'willis', bank=44, stack=333)
sally = Player('sdf9', 'Sally', bank=44, stack=333)
lucy = Player('sdf8', 'Lucy', bank=44, stack=333)
patty = Player('sdf7', 'Patty', bank=44, stack=333)

In [335]:
print(joe)

Joe is the player and they have 333 chips


In [342]:
t = Table(5)

In [349]:
t.addPlayerToTable(joe)
t.addPlayerToTable(fred)
t.addPlayerToTable(willis)
t.addPlayerToTable(sally)
t.addPlayerToTable(lucy)
t.addPlayerToTable(patty)

Table is full


In [142]:
joe.atTable

In [351]:
for player in t.players:
    player.sitDown()

In [264]:
joe.leaveTable()

In [180]:
fred.standUp()

In [350]:
print(t)

Table online is False Players are ['Joe', 'Fred', 'willis', 'Sally', 'Lucy']

 Open Seat nums[0, 1, 2, 3, 4]
seated players:



In [352]:
t.startGame()

In [353]:
t.buttonSeatNum


0

In [268]:
t.getNextSeatNum(3)

0

In [252]:
t.removePlayerFromTable(willis)

In [354]:
t.resetRound()

New Betting Round
Button is at seat Number: 1


AttributeError: 'Player' object has no attribute 'Bet'

In [355]:
t.moveBtton()

In [359]:
t.clearPlayerHands()
t.deal()

Player Lucy hand is ['Kc', '4c']
Player Sally hand is ['Qh', 'Qd']
Player willis hand is ['7d', '8c']
Player Fred hand is ['6s', '7s']
Player Joe hand is ['Jc', '10h']


In [190]:
print(list(range(5)))

[0, 1, 2, 3, 4]


In [ ]:
#init

#create Table
#While Table Online
    #for player in Table.players where player.seated == True
    #create list of seated players
    #get ante
        #update player investment
        #update pot 
    #deal
    #betting round
        #
    #flop
    #betting round
    #turn
    #betting round
    #river
    
    
    
    
    # Update player status (seated or standing)
    # update if table is online or offline



In [ ]:
d.clear()
d.populate()
d.shuffle()

In [ ]:
d.deal(4)

In [227]:
f=[3,4,8,6]

In [228]:
f.index(8)

2

In [331]:
username = input("Enter username:")
print("Username is: " + username)

Enter username: sdfd


Username is: sdfd
